# GPU 训练

```{note}
在 GPU 上训练大语言模型可以显著提高训练速度。
```

## 环境配置

我的系统是 windows 11，显卡是 4060，在此之上我做了如下环境配置：
1. 使用 wsl2 安装 ubuntu 24.04
2. 更新显卡驱动
3. 在 ubuntu 系统下安装 conda，创建了一个新的环境，Python 版本是 3.13.9，安装了对应 PyTorch 版本
4. 为了方便开发，安装了 windows 版本的 Trae，然后用 wsl 连上 ubuntu 系统

In [1]:
import torch

print("PyTorch版本:", torch.__version__)
print("CUDA版本:", torch.version.cuda)  # 关键：这是PyTorch实际使用的CUDA版本
print("cuDNN版本:", torch.backends.cudnn.version())
print("GPU是否可用:", torch.cuda.is_available())
print("GPU数量:", torch.cuda.device_count())
print("当前GPU索引:", torch.cuda.current_device())
print("GPU名称:", torch.cuda.get_device_name(0))

PyTorch版本: 2.10.0+cu130
CUDA版本: 13.0
cuDNN版本: 91501
GPU是否可用: True
GPU数量: 1
当前GPU索引: 0
GPU名称: NVIDIA GeForce RTX 4060


## 使用 GPU 训练

要使用 GPU 训练，需要：
1. 模型放到 GPU 里：`model = SimpleNextTokenModel(vocab_size, embed_dim, hidden_dim).to(device)`
2. 数据放到 GPU 里：训练时 `batch.to(device)`，预测时 `test_input = train_data[0][:, :-1].to(device)`

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNextTokenModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        # 1. Embedding 层
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 2. MLP 层
        # 简单结构: Linear -> Activation -> Linear
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim)
        )
        
        # 3. 输出层 (Linear)
        # 将维度从 embed_dim 映射回 vocab_size
        self.output_head = nn.Linear(embed_dim, vocab_size, bias=False)

    def forward(self, input_ids):
        """
        参数:
            input_ids: (batch_size, sequence_length)
        返回:
            logits: (batch_size, sequence_length, vocab_size)
        """
        # x shape: (batch_size, seq_len, embed_dim)
        x = self.embedding(input_ids)
        
        # x shape: (batch_size, seq_len, embed_dim)
        x = self.mlp(x)
        
        # logits shape: (batch_size, seq_len, vocab_size)
        logits = self.output_head(x)
        
        return logits

def generate_dummy_data(num_batches, batch_size, seq_len, vocab_size):
    """生成简单的随机数据"""
    data = []
    for _ in range(num_batches):
        # 随机生成数据
        batch = torch.randint(0, vocab_size, (batch_size, seq_len))
        data.append(batch)
    return data


# 1. 设置随机种子，保证可复现性
torch.manual_seed(42)

# 自动检测设备
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print(f"使用设备: {device}")

# 2. 超参数
vocab_size = 1000
embed_dim = 256
hidden_dim = 1024
seq_len = 128   # 序列长度
batch_size = 32
learning_rate = 1e-3
num_batches = 100 # 训练步数

# 3. 初始化模型并移动到设备
model = SimpleNextTokenModel(vocab_size, embed_dim, hidden_dim).to(device)
print("模型结构初始化完成。")

# 4. 定义 Loss 和 Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# 5. 准备虚拟数据
# 注意：这里的 data 包含输入和目标，所以 seq_len + 1
train_data = generate_dummy_data(num_batches, batch_size, seq_len + 1, vocab_size)

start_time = time.time()
print(f"开始训练，共 {num_batches} 个 batch...")
model.train()

for step, batch in enumerate(train_data):
    # 将数据移动到设备
    batch = batch.to(device)

    # 构造输入和目标 (Next Token Prediction)
    # 输入: 序列的前 N-1 个 token
    # 目标: 序列的后 N-1 个 token (即每个位置的下一个 token)
    input_ids = batch[:, :-1]  # (B, T)
    targets = batch[:, 1:]     # (B, T)
    
    # 清零梯度
    optimizer.zero_grad()
    
    # 前向传播
    logits = model(input_ids) # (B, T, V)
    
    # 计算 Loss
    # CrossEntropyLoss 需要 (N, C) 和 (N) 的输入
    # logits.view(-1, vocab_size) -> (B*T, V)
    # targets.view(-1) -> (B*T)
    loss = criterion(logits.view(-1, vocab_size), targets.reshape(-1))
    
    # 反向传播
    loss.backward()
    
    # 更新参数
    optimizer.step()
    
    if (step + 1) % 10 == 0:
        print(f"Step [{step+1}/{num_batches}], Loss: {loss.item():.4f}")

print("训练结束！")
end_time = time.time()
print(f"训练耗时: {end_time - start_time:.2f} 秒")

# 简单验证一下
test_input = train_data[0][:, :-1].to(device)
with torch.no_grad():
    logits = model(test_input)
    preds = torch.argmax(logits, dim=-1)
    print("\n验证第一个 batch 的预测:")
    print(f"输入 shape: {test_input.shape}")
    print(f"预测 shape: {preds.shape}")
    # 这里只是随机数据，预测准确率不会高，主要是验证流程跑通



使用设备: cuda
模型结构初始化完成。
开始训练，共 100 个 batch...
Step [10/100], Loss: 6.9229
Step [20/100], Loss: 6.9222
Step [30/100], Loss: 6.9219
Step [40/100], Loss: 6.9241
Step [50/100], Loss: 6.9204
Step [60/100], Loss: 6.9230
Step [70/100], Loss: 6.9176
Step [80/100], Loss: 6.9225
Step [90/100], Loss: 6.9193
Step [100/100], Loss: 6.9180
训练结束！
训练耗时: 0.62 秒

验证第一个 batch 的预测:
输入 shape: torch.Size([32, 128])
预测 shape: torch.Size([32, 128])
